In [ ]:
import os

# Detect if we are in the TIRA sandbox
# Install the required dependencies if we are not in the sandbox.
if 'TIRA_DATASET_ID' not in os.environ:
    !pip3 install python-terrier tira==0.0.88 ir_datasets
else:
    print('We are in the TIRA sandbox.')


In [ ]:
# Import the required libraries
print('importing libraries...')
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
ensure_pyterrier_is_loaded()

from load_dataset import load_dataset
from create_index import create_index
from create_model import create_model
print('Done. Libraries imported.')

In [ ]:
# load data
load_dataset_result = load_dataset()
documents, queries = load_dataset_result['documents'], load_dataset_result['queries']
print("data load")

# create index
index = create_index(documents)
print("index created")

# create model
model = create_model(index)
print("model created")

# run model
run = model(queries)
persist_and_normalize_run(run, 'bm25-baseline')